In [ ]:
!pip install jsonlines
!pip install pymorphy2

In [1]:
import pandas as pd
import jsonlines
import string
import re

import glob
import io
import os
import random

import pymorphy2
m = pymorphy2.MorphAnalyzer()

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

import string
from string import punctuation

from tqdm.notebook import tqdm

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ekostrykina/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
def get_data(filename):
    data = []
    with jsonlines.open(f'TERRa/{filename}.jsonl') as reader:
        for obj in reader:
            data.append(obj)
    return pd.DataFrame(data)

In [3]:
val_df = get_data('val')

In [4]:
val_df

,premise,hypothesis,label,idx
0,"""""""Решение носит символический характер, так к...",Взыскать компенсацию не получится.,entailment,0
1,"Об этом вечером во вторник, 17 января, сообщил...",На улице Кеэвисе жителей эвакуировали из-за мины.,entailment,1
2,"""В суде Центрального района Сочи завершилось р...",81-летний пенсионер застрелил обидчика дочери.,entailment,2
3,"""Кто-то нарисовал на снегу контур человеческог...",Кто-то изобразил на снегу контур человеческого...,entailment,3
4,"Также установлено, что работники учреждения не...",В детском саду не все вовремя проходят медосмотр.,entailment,4
...,...,...,...,...
302,"Грабитель поджидал своих жертв и, угрожая им п...",У грабителя был пистолет.,entailment,302
303,Во время обыска в квартире подозреваемого поли...,С ноября прошлого года подозреваемый был на св...,entailment,303
304,"""""""После инцидента 15 августа 2011 г. ко мне п...",Береговой не сделал видеозапись.,not_entailment,304
305,Сыграло ли это событие роль в решении обвиняем...,Дивенни мог работать на иностранную разведку.,entailment,305


In [ ]:
cases = set(['nomn', # именительный
         'gent', # родительный
         'datv', # дательный
         'accs', # винительный
         'ablt', # творительный
         'loct' # предложный
         ])

In [13]:
tenses = set(['past', 'pres', 'futr'])

In [65]:
def inflect_verb(sentence):
    text = word_tokenize(sentence)
    for idx, word in enumerate(text):
        ok = True
        parsed_word = m.parse(word)[0]
        if parsed_word.tag.POS == 'VERB':
            original_tense = parsed_word[1].tense       
            tenses_available = tenses.copy()
            try:
                tenses_available.remove(original_tense)
            except:
                continue

            tense_to_inflect = random.choice(list(tenses_available))
            tenses_available.remove(tense_to_inflect)
            inflected_word = parsed_word.inflect({tense_to_inflect})
            try:
                inflected_word = inflected_word.word
            except:
                ok = False
                pass
            
            while parsed_word.word == inflected_word and len(tenses_available) > 0:
                tense_to_inflect = random.choice(list(tenses_available))
                tenses_available.remove(tense_to_inflect)

                inflected_word = parsed_word.inflect({tense_to_inflect})
                try:
                    inflected_word = inflected_word.word  
                except:
                    ok = False
                    pass

            if ok == True:
                original_word = text[idx]
                text[idx] = inflected_word
                text = ' '.join(text)
                # удаляем лишние пробелы
                pat = "\s+([{}]+)".format(re.escape(string.punctuation))
                output_text = re.sub("\s{2,}", " ", re.sub(pat, r"\1", text))
                return output_text, True, original_word, inflected_word           
        
    return sentence, False, None, None

In [19]:
inflect_verb('падал')

('падаю', True, 'падал', 'падаю')

In [22]:
inflect_verb('падает')

('падал', True, 'падает', 'падал')

In [47]:
inflect_verb('падает, купается')

падает futr
купается past


('падает, купался', True, 'купается', 'купался')

In [66]:
attacked_data = []
flags = []
original_words = []
inflected_words = []

for i in tqdm(val_df['hypothesis']):
    sentence, flag, original_word, inflected_word = inflect_verb(i)
    attacked_data.append(sentence)
    flags.append(flag)
    original_words.append(original_word)
    inflected_words.append(inflected_word)

  0%|          | 0/307 [00:00<?, ?it/s]

In [68]:
val_df['original_hypothesis'] = val_df['hypothesis']
val_df['hypothesis'] = attacked_data
val_df['flags'] = flags
val_df['original_words'] = original_words
val_df['inflected_words'] = inflected_words

In [69]:
val_df

,premise,hypothesis,label,idx,original_hypothesis,flags,original_words,inflected_words
0,"""""""Решение носит символический характер, так к...",Взыскать компенсацию не получится.,entailment,0,Взыскать компенсацию не получится.,False,None,None
1,"Об этом вечером во вторник, 17 января, сообщил...",На улице Кеэвисе жителей эвакуируем из-за мины.,entailment,1,На улице Кеэвисе жителей эвакуировали из-за мины.,True,эвакуировали,эвакуируем
2,"""В суде Центрального района Сочи завершилось р...",81-летний пенсионер застрелил обидчика дочери.,entailment,2,81-летний пенсионер застрелил обидчика дочери.,False,None,None
3,"""Кто-то нарисовал на снегу контур человеческог...",Кто-то изобразил на снегу контур человеческого...,entailment,3,Кто-то изобразил на снегу контур человеческого...,False,None,None
4,"Также установлено, что работники учреждения не...",В детском саду не все вовремя проходят медосмотр.,entailment,4,В детском саду не все вовремя проходят медосмотр.,False,None,None
...,...,...,...,...,...,...,...,...
302,"Грабитель поджидал своих жертв и, угрожая им п...",У грабителя есть пистолет.,entailment,302,У грабителя был пистолет.,True,был,есть
303,Во время обыска в квартире подозреваемого поли...,С ноября прошлого года подозреваемый есть на с...,entailment,303,С ноября прошлого года подозреваемый был на св...,True,был,есть
304,"""""""После инцидента 15 августа 2011 г. ко мне п...",Береговой не сделаю видеозапись.,not_entailment,304,Береговой не сделал видеозапись.,True,сделал,сделаю
305,Сыграло ли это событие роль в решении обвиняем...,Дивенни могу работать на иностранную разведку.,entailment,305,Дивенни мог работать на иностранную разведку.,True,мог,могу


In [70]:
val_df.to_csv('attack4_val.csv', index=False)